그룹 연산
- 복잡한 데이터를 어떤 기준에 따라 여러 그룹으로 나누어서 관찰할 수 있으며 이런 방식으로 분할 처리하는 것을 그룹 연산이라 함
- 그룹 연산은 데이터를 집계, 변환, 필터링하는데 효율적이며 pandas groupby() 메소드를 사용함
- 그룹 객체 만들기 (분할)
- 그룹 연산 메소드(적용,결합)

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

titanic = sns.load_dataset('titanic')

df = titanic.loc[:,['age','sex','class','fare','survived']]
print('승객 수: ',len(df))
df.head()

승객 수:  891


,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [6]:
df.value_counts('class')

class
Third     491
First     216
Second    184
dtype: int64

In [8]:
grouped = df.groupby(['class'])
#list(grouped)

for key ,group in grouped:
    print('* key:' ,key)
    print('* number:', len(group))
    print(group.head())

* key: First
* number: 216
     age     sex  class     fare  survived
1   38.0  female  First  71.2833         1
3   35.0  female  First  53.1000         1
6   54.0    male  First  51.8625         0
11  58.0  female  First  26.5500         1
23  28.0    male  First  35.5000         1
* key: Second
* number: 184
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
17   NaN    male  Second  13.0000         1
20  35.0    male  Second  26.0000         0
21  34.0    male  Second  13.0000         1
* key: Third
* number: 491
    age     sex  class     fare  survived
0  22.0    male  Third   7.2500         0
2  26.0  female  Third   7.9250         1
4  35.0    male  Third   8.0500         0
5   NaN    male  Third   8.4583         0
7   2.0    male  Third  21.0750         0


In [10]:
average = grouped.mean()
average

,age,fare,survived
class,,,
First,38.233441,84.154687,0.629630
Second,29.877630,20.662183,0.472826
Third,25.140620,13.675550,0.242363


In [11]:
grouped.max()

,age,sex,fare,survived
class,,,,
First,80.0,male,512.3292,1
Second,70.0,male,73.5000,1
Third,74.0,male,69.5500,1


In [22]:
# 'third' 그룹만을 선택해서 group3으로 저장하고 통계요약표 출력 > describe()
group3= df.loc[df['class']=='Third']
group3.describe().round(4)

,age,fare,survived
count,355.0000,491.0000,491.0000
mean,25.1406,13.6756,0.2424
std,12.4954,11.7781,0.4289
min,0.4200,0.0000,0.0000
25%,18.0000,7.7500,0.0000
50%,24.0000,8.0500,0.0000
75%,32.0000,15.5000,0.0000
max,74.0000,69.5500,1.0000


In [21]:
group33 = grouped.get_group('Third')
group33.describe().round(4)

,age,fare,survived
count,355.0000,491.0000,491.0000
mean,25.1406,13.6756,0.2424
std,12.4954,11.7781,0.4289
min,0.4200,0.0000,0.0000
25%,18.0000,7.7500,0.0000
50%,24.0000,8.0500,0.0000
75%,32.0000,15.5000,0.0000
max,74.0000,69.5500,1.0000


In [25]:
# class열, sex  열 기준으로 분할하여 grouped_two에 저장
grouped_two = df.groupby(['class','sex'])
for key,group in grouped_two:
    print('* key:', key)
    print('* number:', len(group))
    print(group.head())

* key: ('First', 'female')
* number: 94
     age     sex  class      fare  survived
1   38.0  female  First   71.2833         1
3   35.0  female  First   53.1000         1
11  58.0  female  First   26.5500         1
31   NaN  female  First  146.5208         1
52  49.0  female  First   76.7292         1
* key: ('First', 'male')
* number: 122
     age   sex  class      fare  survived
6   54.0  male  First   51.8625         0
23  28.0  male  First   35.5000         1
27  19.0  male  First  263.0000         0
30  40.0  male  First   27.7208         0
34  28.0  male  First   82.1708         0
* key: ('Second', 'female')
* number: 76
     age     sex   class     fare  survived
9   14.0  female  Second  30.0708         1
15  55.0  female  Second  16.0000         1
41  27.0  female  Second  21.0000         0
43   3.0  female  Second  41.5792         1
53  29.0  female  Second  26.0000         1
* key: ('Second', 'male')
* number: 108
     age   sex   class  fare  survived
17   NaN  male  Secon

In [26]:
average_two = grouped_two.mean()
average_two

age        fare  survived
class  sex                                    
First  female  34.611765  106.125798  0.968085
       male    41.281386   67.226127  0.368852
Second female  28.722973   21.970121  0.921053
       male    30.740707   19.741782  0.157407
Third  female  21.750000   16.118810  0.500000
       male    26.507589   12.661633  0.135447

In [31]:
# ('Third','Female')선택해서 group3f에 저장하고 처음 5개행을 출력하세요
group3f = grouped_two.get_group(('Third','female'))
group3f.describe()

,age,fare,survived
count,102.000000,144.000000,144.000000
mean,21.750000,16.118810,0.500000
std,12.729964,11.690314,0.501745
min,0.750000,6.750000,0.000000
25%,14.125000,7.854200,0.000000
50%,21.500000,12.475000,0.500000
75%,29.750000,20.221875,1.000000
max,63.000000,69.550000,1.000000


적용 - 결합
데이터 집계 (agg)
- 집계 연산을 처리하는 사용자 정의함수를 그룹 객체에 적용하려면 agg() 메소드 사용
- 모든 열에 여러 함수를 매핑: group객체.agg([함수1,함수2,함수3,...])
- 각 열마다 다른 함수를 매핑: group객체.agg({'열1':함수1,'열2':함수2,...}) >dict

변환(transform)
- 원소의 본래 행 인덱스와 열 이름을 기준으로 연산 결과를 반환
- 데이터 변환 연산 : group객체.transform(매핑 함수)

필터링(filter)
- 그룹 객체에 filter() 메소드를 적용할 때 조건식을 가진 함수를 전달하면 조건이 참인 그룹만을 남김
- 그룹 객체 필터링: group객체.filter(조건 함수식)

객체에 함수 매핑(apply)
- pandas 객체의 개별 원소를 특정 함수에 일대일로 매핑. 사용자가 원하는 대부분의 연산을 그룹 객체에 적용
- group객체.apply(매핑함수)

In [85]:
# (최대값 - 최소값)을 반환하는 사용자 함수를 정의하고 agg() 함수에 인수로 전달하여 그룹별로 집계하여 agg_minmax 이름으로 저장후 처음 5개행을 출력하세요
def min_max(x):
    return x.max()-x.min()
    
    
titanic = sns.load_dataset('titanic')
df = titanic.loc[:,['age','sex','class','fare','survived']]
grouped = df.groupby(['class'])

grouped.agg(min_max)

C:\Users\admin\anaconda3\envs\cakd5\lib\site-packages\pandas\core\groupby\generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,age,fare,survived
class,,,
First,79.08,512.3292,1
Second,69.33,73.5000,1
Third,73.58,69.5500,1


In [48]:
agg_all = grouped.agg(['min','max'])
agg_all.head()

age           sex       fare           survived    
         min   max     min   max  min       max      min max
class                                                       
First   0.92  80.0  female  male  0.0  512.3292        0   1
Second  0.67  70.0  female  male  0.0   73.5000        0   1
Third   0.42  74.0  female  male  0.0   69.5500        0   1

In [51]:
agg_any = grouped.agg({'age':'mean','fare':['min','max']})
agg_any

age fare          
             mean  min       max
class                           
First   38.233441  0.0  512.3292
Second  29.877630  0.0   73.5000
Third   25.140620  0.0   69.5500

In [90]:
filter200 = grouped.filter(lambda x: len(x)>=200)
filter200.value_counts('class')

class
Third     491
First     216
Second      0
dtype: int64

In [95]:
age_filter = grouped.filter(lambda x : x['age'].mean() <30)
age_filter.tail()

,age,sex,class,fare,survived
884,25.0,male,Third,7.050,0
885,39.0,female,Third,29.125,0
886,27.0,male,Second,13.000,0
888,NaN,female,Third,23.450,0
890,32.0,male,Third,7.750,0


In [96]:
for key,group in grouped:
    print('* key:', key)
    print('* number:', len(group))
    print(group.describe())

* key: First
* number: 216
              age        fare    survived
count  186.000000  216.000000  216.000000
mean    38.233441   84.154687    0.629630
std     14.802856   78.380373    0.484026
min      0.920000    0.000000    0.000000
25%     27.000000   30.923950    0.000000
50%     37.000000   60.287500    1.000000
75%     49.000000   93.500000    1.000000
max     80.000000  512.329200    1.000000
* key: Second
* number: 184
              age        fare    survived
count  173.000000  184.000000  184.000000
mean    29.877630   20.662183    0.472826
std     14.001077   13.417399    0.500623
min      0.670000    0.000000    0.000000
25%     23.000000   13.000000    0.000000
50%     29.000000   14.250000    0.000000
75%     36.000000   26.000000    1.000000
max     70.000000   73.500000    1.000000
* key: Third
* number: 491
              age        fare    survived
count  355.000000  491.000000  491.000000
mean    25.140620   13.675550    0.242363
std     12.495398   11.778142    0.4

In [100]:
agg_grouped = grouped.apply(lambda x:x.describe())
agg_grouped

age        fare    survived
class                                           
First  count  186.000000  216.000000  216.000000
       mean    38.233441   84.154687    0.629630
       std     14.802856   78.380373    0.484026
       min      0.920000    0.000000    0.000000
       25%     27.000000   30.923950    0.000000
       50%     37.000000   60.287500    1.000000
       75%     49.000000   93.500000    1.000000
       max     80.000000  512.329200    1.000000
Second count  173.000000  184.000000  184.000000
       mean    29.877630   20.662183    0.472826
       std     14.001077   13.417399    0.500623
       min      0.670000    0.000000    0.000000
       25%     23.000000   13.000000    0.000000
       50%     29.000000   14.250000    0.000000
       75%     36.000000   26.000000    1.000000
       max     70.000000   73.500000    1.000000
Third  count  355.000000  491.000000  491.000000
       mean    25.140620   13.675550    0.242363
       std     12.495398   11.778142    0.428949
       min      0.420000    0.000000    0.000000
       25%     18.000000    7.750000    0.000000
       50%     24.000000    8.050000    0.000000
       75%     32.000000   15.500000    0.000000
       max     74.000000   69.550000    1.000000

In [118]:
def distance(x):
    return (x-x.mean())/x.std()

agg_distance = grouped.age.apply(lambda x : distance(x))
agg_distance


0     -0.251342
1     -0.015770
2      0.068776
3     -0.218434
4      0.789041
         ...   
886   -0.205529
887   -1.299306
888         NaN
889   -0.826424
890    0.548953
Name: age, Length: 891, dtype: float64

In [111]:
agg_distance2 = grouped.age.transform(lambda x : distance(x))
agg_distance2

0     -0.251342
1     -0.015770
2      0.068776
3     -0.218434
4      0.789041
         ...   
886   -0.205529
887   -1.299306
888         NaN
889   -0.826424
890    0.548953
Name: age, Length: 891, dtype: float64

In [121]:
df1 = df.loc[df['class']=='First']
df1

,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
6,54.0,male,First,51.8625,0
11,58.0,female,First,26.5500,1
23,28.0,male,First,35.5000,1
...,...,...,...,...,...
871,47.0,female,First,52.5542,1
872,33.0,male,First,5.0000,0
879,56.0,female,First,83.1583,1
887,19.0,female,First,30.0000,1


In [126]:
gr1 = df.groupby(['class'])
print(gr1.get_group('First'))

      age     sex  class     fare  survived
1    38.0  female  First  71.2833         1
3    35.0  female  First  53.1000         1
6    54.0    male  First  51.8625         0
11   58.0  female  First  26.5500         1
23   28.0    male  First  35.5000         1
..    ...     ...    ...      ...       ...
871  47.0  female  First  52.5542         1
872  33.0    male  First   5.0000         0
879  56.0  female  First  83.1583         1
887  19.0  female  First  30.0000         1
889  26.0    male  First  30.0000         1

[216 rows x 5 columns]


In [122]:
df1f = df1.loc[df['sex']=='female']
df1f

,age,sex,class,fare,survived
1,38.0,female,First,71.2833,1
3,35.0,female,First,53.1000,1
11,58.0,female,First,26.5500,1
31,NaN,female,First,146.5208,1
52,49.0,female,First,76.7292,1
...,...,...,...,...,...
856,45.0,female,First,164.8667,1
862,48.0,female,First,25.9292,1
871,47.0,female,First,52.5542,1
879,56.0,female,First,83.1583,1


In [128]:
gr2 = df.groupby(['class','sex'])
print(gr2.get_group(('First','female')))

      age     sex  class      fare  survived
1    38.0  female  First   71.2833         1
3    35.0  female  First   53.1000         1
11   58.0  female  First   26.5500         1
31    NaN  female  First  146.5208         1
52   49.0  female  First   76.7292         1
..    ...     ...    ...       ...       ...
856  45.0  female  First  164.8667         1
862  48.0  female  First   25.9292         1
871  47.0  female  First   52.5542         1
879  56.0  female  First   83.1583         1
887  19.0  female  First   30.0000         1

[94 rows x 5 columns]


피벗
- 피벗 테이블을 구성하는 4가지 요소(행 인덱스, 열 인덱스, 데이터 값, 데이터 집계함수)에 적용할 데이터프레임의 열을 각각 지정하여 함수의 인자로 전달